In [ ]:
# 1차적으로 git clone을 진행한다.
# ultralytics yolov5를 활용.
!git clone https://github.com/ultralytics/yolov5
%pip install -qr requriements.txt

In [1]:
# 필요한 module upload
import torch
# yaml 파일을 만들기 위한 module
import yaml
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import cv2
# json file
import json
import os
# XML 어노테이션 파일
import xml.etree.ElementTree as ET
# 이미지를 한 번에 긁어모으기 위한 glob 모듈
from glob import glob
# 파일 이동
import shutil
# Image를 open
from PIL import Image
# plt.imshow()후 커널 dead 현상 제거
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [2]:
# cuda 연산
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
%cd "C:\\Users\\user\\Desktop\\recognition_project\\yolov5"

C:\Users\user\Desktop\recognition_project\yolov5


In [4]:
pwd

'C:\\Users\\user\\Desktop\\recognition_project\\yolov5'

In [5]:
# 우선 file을 train과 valid, test로 나눠준다
# directory 생성 후 나중에 상황에 맞게 분리 해준다
# path = "./archive"
# img_path = path+"\\"+"images"
# annot_path = path+"\\"+"annotations"
# os.mkdir(img_path+"\\"+"train")
# os.mkdir(img_path+"\\"+"valid")
# os.mkdir(img_path+"\\"+"test")
# os.mkdir(annot_path+"\\"+"train")
# os.mkdir(annot_path+"\\"+"valid")
# os.mkdir(annot_path+"\\"+"test")

In [6]:
# yolov5 model load
# 우선은 따로 구축하진 않고 있는 parameter 활용
# parameter 활용 시 customizing 하는 방법 생각
model = torch.hub.load("ultralytics/yolov5", "yolov5s")

Using cache found in C:\Users\user/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-3-30 Python-3.9.13 torch-1.13.1+cu116 CUDA:0 (GeForce RTX 3060 Laptop GPU, 6144MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


우선 kaggle의 데이터셋에는 annotation file까지 준비가 된 상태이다.
1. 데이터 파일을 coco 형식으로 만들어 주거나 또는 이미 xml로 annotation 되어 있다.
    - 우선 이미지와 객체에 대한 label 정보를 포함하는 JSON 파일을 생성한다
    - 즉, 이미지와 객체에 대한 label 정보를 포함하는 JSON 파일을 이용하여 이미지와 라벨을 저장한다.
2. 'data.yaml' 파일 생성
    - 클래스 개수와 클래스 이름 설정
        - coco 형식으로 구성된 데이터셋에 대한 클래스 개수와 클래스 이름을 설정한다. 이 정보는 data.yaml에 포함된다
    - 학습/검증 데이터 경로 설정
        - 학습과 검증 데이터 경로를 설정. 이 경로 정보는 data.yaml 파일에 포함
    - data.yaml 파일 생성
        - 설정된 정보를 이용해 data.yaml 파일을 생성. 생성된 파일은 coco 데이터셋을 이용하는 YOLOv5 모델에서 학습에 사용된다.


In [7]:
# xml 파일의 구성이 어떻게 돼 있는지 확인
xml_file = "./archive/annotations/train/hard_hat_workers1.xml"
# xml 파일 열기(parsing)
tree = ET.parse(xml_file)
# xml 파일의 root 요소 가져오기
root = tree.getroot()
# root 요소의 tag 이름 출력
print(root.tag)
# 루트 요소의 자식 요소 출력
for child in root:
    print(child.tag, child.attrib)

annotation
folder {}
filename {}
size {}
segmented {}
object {}
object {}
object {}
object {}
object {}
object {}
object {}
object {}
object {}


다음의 아래의 코드는 annotation된 것을 YOLO에 맞게 변환 시켜준 것이다.
- COCO 형식이 아닌 PASCAL VOC형태.
- XML 형태의 ANNOTATION 파일을 YOLO 형식에 맞게

In [8]:
# YOLO 형식으로 변환할 class 이름을 지정
classes = ["helmet", "head", "person"]
# xml 파일과 이미지 파일이 저장된 폴더 경로를 지정
# 각 train, test, valid 별로 진행해 준다.
types = ["train", "valid", "test"]
# yolo 형식으로 변환된 파일을 저장할 폴더를 생성
%cd "C:\\Users\\user\\Desktop\\recognition_project\\yolov5\\archive"
if not os.path.exists("labels"):
    os.makedirs("labels")
    %cd "C:\\Users\\user\\Desktop\\recognition_project\\yolov5\\archive\\labels"
    os.makedirs("train")
    os.makedirs("valid")
    os.makedirs("test")
    %cd "C:\\Users\\user\\Desktop\\recognition_project\\yolov5\\archive"
# xml 파일을 parsing해서 yolo 형식으로 변환
# os.listdir -> xml 데이터들이 담겨 있다.
cnt = 0
for t in types:
    xml_path = "./annotations/" + t
    image_path = "./images/" + t
    for xml_file_name in os.listdir(xml_path):
        # ./annotations/train\\*.xml
        xml_file_path = os.path.join(xml_path, xml_file_name)
        # xml 파일을 parsing
        tree = ET.parse(xml_file_path)
        root = tree.getroot()
        # 이미지 파일 경로 가져오기
        # image_file_name에는 해당 xml에 대응되는 이미지 파일의 이름을 나타낸다.
        image_file_name = root.find("filename").text
        image_file_path = os.path.join(image_path, image_file_name)
        # 이미지 파일 열기
        img = Image.open(image_file_path)
        img_width, img_height = img.size # 폭 x 높이
        # YOLO 형식으로 변환 단계
        # xml_file_name[:-4] 에는 .xml 전까지의 파일 명까지만 cut
        # .txt형태로 변환
        # 각 directory별로 들어가게 된다.
        yolo_file_path = os.path.join("./labels",t,xml_file_name[:-4] + ".txt")
        with open(yolo_file_path, "w") as f:
            # 해당 xml 파일에서 <object> element를 찾아서 반복적으로 처리하는 코드.
            # <object> element는 객체의 정보를 포함하고 있다.
            # 각 객체의 정보를 추출한다. 추출된 정보를 사용하여 YOLO 형식으로 변환한 뒤, 텍스트 파일에 저장
            # 반복문을 통해 object를 하나씩 가져온다.
            for obj in root.findall("object"):
                # 각 객체 클래스의 클래스 이름을 문자열로 저장
                # class명을 하나씩 가져온다
                class_name = obj.find("name").text
                # 해당 cell의 맨 위에 classes를 정의하였다.
                # classes.index(class_name) 이라고 하면 내가 정의한 list의 순서대로 index를 반환한다.
                class_id = classes.index(class_name)

                # 해당 <object> element에서 하위 element인 bndbox element를 찾아서 반환한다.
                bbox = obj.find("bndbox")
                x_min = int(bbox.find("xmin").text)
                y_min = int(bbox.find("ymin").text)
                x_max = int(bbox.find("xmax").text)
                y_max = int(bbox.find("ymax").text)

                # normalize 시켜주는 과정
                # 1.0은 이미지의 너비 또는 높이를 1로 정규화 하는데 사용
                dx = np.float32(1./int(img.size[0]))
                dy = np.float32(1./int(img.size[1]))
                # x_center는 해당 객체의 bounding box 중심점의 x좌표
                # 값을 YOLO 형식으로 bounding box 좌표를 변환하기 위해 사용
                # 0 ~ 1 사이로 정규화 되어 있으므로 이 값을 YOLO 형식으로 변환 시에도 0 ~ 1
                # y_center도 마찬가지의 원리
                # x_center, y_center, w, h는 YOLO 형식으로 bounding box 좌표를 나타내는 값
                # YOLO: 이미지를 Grid로 나누고 각 grid 셀마다 bounding box를 예측
                # bounding box는 해당 셀에 대해 상대적인 위치와 크기로 표현
                # 이를 위해 x_center, y_center, w, h를 사용한다
                # x_center: bounding box 중심점의 x 좌표
                x_center = (x_min+x_max) / 2 * dx
                # y_center: bounding box의 중심점의 y좌표
                y_center = (y_min+y_max) / 2 * dy
                # bounding box의 너비(해당 셀의 너비를 기준으로 상대적인 크기)
                w = (x_max - x_min) / img_width
                # bounding box의 높이(해당 셀의 높이를 기준으로 한 상대적인 크기)
                h = (y_max - y_min) / img_height

                # YOLO 형식으로 변환된 Bounding box 정보를 txt 파일에 기록
                # class_id: 해당 객체의 class id를 나타낸다.
                # classes에 설정된 class 명의 index를 할당
                # helmet, head, person의 클래스 ID를 할당
                # x_center, y_center, w, h는 YOLO 형식으로 bounding box 좌표를 나타낸다.
                f.write(f"{class_id} {x_center:.5f} {y_center:.5f} {w:.5f} {h:.5f}\n")
            print("Convert these files, {} file and {} file, into the YOLO format".format(xml_file_name, image_file_name))

C:\Users\user\Desktop\recognition_project\yolov5\archive
C:\Users\user\Desktop\recognition_project\yolov5\archive\labels
C:\Users\user\Desktop\recognition_project\yolov5\archive
Convert these files, hard_hat_workers0.xml file and hard_hat_workers0.png file, into the YOLO format
Convert these files, hard_hat_workers1.xml file and hard_hat_workers1.png file, into the YOLO format
Convert these files, hard_hat_workers10.xml file and hard_hat_workers10.png file, into the YOLO format
Convert these files, hard_hat_workers100.xml file and hard_hat_workers100.png file, into the YOLO format
Convert these files, hard_hat_workers1000.xml file and hard_hat_workers1000.png file, into the YOLO format
Convert these files, hard_hat_workers1001.xml file and hard_hat_workers1001.png file, into the YOLO format
Convert these files, hard_hat_workers1002.xml file and hard_hat_workers1002.png file, into the YOLO format
Convert these files, hard_hat_workers1003.xml file and hard_hat_workers1003.png file, into 

Convert these files, hard_hat_workers1081.xml file and hard_hat_workers1081.png file, into the YOLO format
Convert these files, hard_hat_workers1082.xml file and hard_hat_workers1082.png file, into the YOLO format
Convert these files, hard_hat_workers1083.xml file and hard_hat_workers1083.png file, into the YOLO format
Convert these files, hard_hat_workers1084.xml file and hard_hat_workers1084.png file, into the YOLO format
Convert these files, hard_hat_workers1085.xml file and hard_hat_workers1085.png file, into the YOLO format
Convert these files, hard_hat_workers1086.xml file and hard_hat_workers1086.png file, into the YOLO format
Convert these files, hard_hat_workers1087.xml file and hard_hat_workers1087.png file, into the YOLO format
Convert these files, hard_hat_workers1088.xml file and hard_hat_workers1088.png file, into the YOLO format
Convert these files, hard_hat_workers1089.xml file and hard_hat_workers1089.png file, into the YOLO format
Convert these files, hard_hat_workers

Convert these files, hard_hat_workers1157.xml file and hard_hat_workers1157.png file, into the YOLO format
Convert these files, hard_hat_workers1158.xml file and hard_hat_workers1158.png file, into the YOLO format
Convert these files, hard_hat_workers1159.xml file and hard_hat_workers1159.png file, into the YOLO format
Convert these files, hard_hat_workers116.xml file and hard_hat_workers116.png file, into the YOLO format
Convert these files, hard_hat_workers1160.xml file and hard_hat_workers1160.png file, into the YOLO format
Convert these files, hard_hat_workers1161.xml file and hard_hat_workers1161.png file, into the YOLO format
Convert these files, hard_hat_workers1162.xml file and hard_hat_workers1162.png file, into the YOLO format
Convert these files, hard_hat_workers1163.xml file and hard_hat_workers1163.png file, into the YOLO format
Convert these files, hard_hat_workers1164.xml file and hard_hat_workers1164.png file, into the YOLO format
Convert these files, hard_hat_workers11

Convert these files, hard_hat_workers1234.xml file and hard_hat_workers1234.png file, into the YOLO format
Convert these files, hard_hat_workers1235.xml file and hard_hat_workers1235.png file, into the YOLO format
Convert these files, hard_hat_workers1236.xml file and hard_hat_workers1236.png file, into the YOLO format
Convert these files, hard_hat_workers1237.xml file and hard_hat_workers1237.png file, into the YOLO format
Convert these files, hard_hat_workers1238.xml file and hard_hat_workers1238.png file, into the YOLO format
Convert these files, hard_hat_workers1239.xml file and hard_hat_workers1239.png file, into the YOLO format
Convert these files, hard_hat_workers124.xml file and hard_hat_workers124.png file, into the YOLO format
Convert these files, hard_hat_workers1240.xml file and hard_hat_workers1240.png file, into the YOLO format
Convert these files, hard_hat_workers1241.xml file and hard_hat_workers1241.png file, into the YOLO format
Convert these files, hard_hat_workers12

Convert these files, hard_hat_workers1309.xml file and hard_hat_workers1309.png file, into the YOLO format
Convert these files, hard_hat_workers131.xml file and hard_hat_workers131.png file, into the YOLO format
Convert these files, hard_hat_workers1310.xml file and hard_hat_workers1310.png file, into the YOLO format
Convert these files, hard_hat_workers1311.xml file and hard_hat_workers1311.png file, into the YOLO format
Convert these files, hard_hat_workers1312.xml file and hard_hat_workers1312.png file, into the YOLO format
Convert these files, hard_hat_workers1313.xml file and hard_hat_workers1313.png file, into the YOLO format
Convert these files, hard_hat_workers1314.xml file and hard_hat_workers1314.png file, into the YOLO format
Convert these files, hard_hat_workers1315.xml file and hard_hat_workers1315.png file, into the YOLO format
Convert these files, hard_hat_workers1316.xml file and hard_hat_workers1316.png file, into the YOLO format
Convert these files, hard_hat_workers13

...

Convert these files, hard_hat_workers4602.xml file and hard_hat_workers4602.png file, into the YOLO format
Convert these files, hard_hat_workers4603.xml file and hard_hat_workers4603.png file, into the YOLO format
Convert these files, hard_hat_workers4604.xml file and hard_hat_workers4604.png file, into the YOLO format
Convert these files, hard_hat_workers4605.xml file and hard_hat_workers4605.png file, into the YOLO format
Convert these files, hard_hat_workers4606.xml file and hard_hat_workers4606.png file, into the YOLO format
Convert these files, hard_hat_workers4607.xml file and hard_hat_workers4607.png file, into the YOLO format
Convert these files, hard_hat_workers4608.xml file and hard_hat_workers4608.png file, into the YOLO format
Convert these files, hard_hat_workers4609.xml file and hard_hat_workers4609.png file, into the YOLO format
Convert these files, hard_hat_workers4610.xml file and hard_hat_workers4610.png file, into the YOLO format
Convert these files, hard_hat_workers

Convert these files, hard_hat_workers4680.xml file and hard_hat_workers4680.png file, into the YOLO format
Convert these files, hard_hat_workers4681.xml file and hard_hat_workers4681.png file, into the YOLO format
Convert these files, hard_hat_workers4682.xml file and hard_hat_workers4682.png file, into the YOLO format
Convert these files, hard_hat_workers4683.xml file and hard_hat_workers4683.png file, into the YOLO format
Convert these files, hard_hat_workers4684.xml file and hard_hat_workers4684.png file, into the YOLO format
Convert these files, hard_hat_workers4685.xml file and hard_hat_workers4685.png file, into the YOLO format
Convert these files, hard_hat_workers4686.xml file and hard_hat_workers4686.png file, into the YOLO format
Convert these files, hard_hat_workers4687.xml file and hard_hat_workers4687.png file, into the YOLO format
Convert these files, hard_hat_workers4688.xml file and hard_hat_workers4688.png file, into the YOLO format
Convert these files, hard_hat_workers

Convert these files, hard_hat_workers4762.xml file and hard_hat_workers4762.png file, into the YOLO format
Convert these files, hard_hat_workers4763.xml file and hard_hat_workers4763.png file, into the YOLO format
Convert these files, hard_hat_workers4764.xml file and hard_hat_workers4764.png file, into the YOLO format
Convert these files, hard_hat_workers4765.xml file and hard_hat_workers4765.png file, into the YOLO format
Convert these files, hard_hat_workers4766.xml file and hard_hat_workers4766.png file, into the YOLO format
Convert these files, hard_hat_workers4767.xml file and hard_hat_workers4767.png file, into the YOLO format
Convert these files, hard_hat_workers4768.xml file and hard_hat_workers4768.png file, into the YOLO format
Convert these files, hard_hat_workers4769.xml file and hard_hat_workers4769.png file, into the YOLO format
Convert these files, hard_hat_workers4770.xml file and hard_hat_workers4770.png file, into the YOLO format
Convert these files, hard_hat_workers

Convert these files, hard_hat_workers4840.xml file and hard_hat_workers4840.png file, into the YOLO format
Convert these files, hard_hat_workers4841.xml file and hard_hat_workers4841.png file, into the YOLO format
Convert these files, hard_hat_workers4842.xml file and hard_hat_workers4842.png file, into the YOLO format
Convert these files, hard_hat_workers4843.xml file and hard_hat_workers4843.png file, into the YOLO format
Convert these files, hard_hat_workers4844.xml file and hard_hat_workers4844.png file, into the YOLO format
Convert these files, hard_hat_workers4845.xml file and hard_hat_workers4845.png file, into the YOLO format
Convert these files, hard_hat_workers4846.xml file and hard_hat_workers4846.png file, into the YOLO format
Convert these files, hard_hat_workers4847.xml file and hard_hat_workers4847.png file, into the YOLO format
Convert these files, hard_hat_workers4848.xml file and hard_hat_workers4848.png file, into the YOLO format
Convert these files, hard_hat_workers

Convert these files, hard_hat_workers4935.xml file and hard_hat_workers4935.png file, into the YOLO format
Convert these files, hard_hat_workers4936.xml file and hard_hat_workers4936.png file, into the YOLO format
Convert these files, hard_hat_workers4937.xml file and hard_hat_workers4937.png file, into the YOLO format
Convert these files, hard_hat_workers4938.xml file and hard_hat_workers4938.png file, into the YOLO format
Convert these files, hard_hat_workers4939.xml file and hard_hat_workers4939.png file, into the YOLO format
Convert these files, hard_hat_workers4940.xml file and hard_hat_workers4940.png file, into the YOLO format
Convert these files, hard_hat_workers4941.xml file and hard_hat_workers4941.png file, into the YOLO format
Convert these files, hard_hat_workers4942.xml file and hard_hat_workers4942.png file, into the YOLO format
Convert these files, hard_hat_workers4943.xml file and hard_hat_workers4943.png file, into the YOLO format
Convert these files, hard_hat_workers

In [9]:
# yaml 파일 제작
# train, test, valid, number of classes, classes name 지정
%cd "C:\\Users\\user\\Desktop\\recognition_project\\yolov5\\archive"
data = {'train': "./archive/images/train",
        'test': "./archive/images/test",
        "val": "./archive/images/valid",
        "nc": "3",
        "names": ['helmet', 'head', 'person']}

# data.yaml은 위의 dictionary 변수 명과 같이 한다.
with open("data.yaml", "w") as f:
    yaml.dump(data, f)

C:\Users\user\Desktop\recognition_project\yolov5\archive


In [10]:
%cd "C:\\Users\\user\\Desktop\\recognition_project\\yolov5"

C:\Users\user\Desktop\recognition_project\yolov5


In [11]:
pwd

'C:\\Users\\user\\Desktop\\recognition_project\\yolov5'

In [15]:
# 추후에 프로젝트 시작 후에 train, validation, test 진행
# 참고
!python detect.py --source "./archive/images/test"

detect: weights=yolov5s.pt, source=./archive/images/test, data=data\coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs\detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5  v7.0-133-gcca5e21 Python-3.9.13 torch-1.13.1+cu116 CUDA:0 (GeForce RTX 3060 Laptop GPU, 6144MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
image 1/1000 C:\Users\user\Desktop\recognition_project\yolov5\archive\images\test\hard_hat_workers4000.png: 640x640 4 persons, 5.0ms
image 2/1000 C:\Users\user\Desktop\recognition_project\yolov5\archive\images\test\hard_hat_workers4001.png: 640x640 8 persons, 15.6ms
image 3/1000 C:\Users\user\Desktop\recognition_project\yolov5\archive\images\test\hard_hat_work

detect: weights=yolov5s.pt, source=./archive/images/train, data=data\coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs\detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5  v7.0-133-gcca5e21 Python-3.9.13 torch-1.13.1+cu116 CUDA:0 (GeForce RTX 3060 Laptop GPU, 6144MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
image 1/3000 C:\Users\user\Desktop\recognition_project\yolov5\archive\images\train\hard_hat_workers0.png: 640x640 12 persons, 0.0ms
image 2/3000 C:\Users\user\Desktop\recognition_project\yolov5\archive\images\train\hard_hat_workers1.png: 640x640 3 persons, 1 bear, 1 skis, 15.6ms
image 3/3000 C:\Users\user\Desktop\recognition_project\yolov5\archive\images\trai